## Main code for Kaggle - Optiver Realized Volatility Prediction
@LaurentMombaerts 13/07/2021

In [2]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


**Lib Import / Data loading**

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import glob
from sklearn.metrics import r2_score

# Load dataset
train = pd.read_csv(r'C:\Users\laurent\ownCloud\Data\Kaggle\optiver-realized-volatility-prediction\train.csv') # Comment the local path for online submission
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train = train[['row_id','target']]

**Functions**

In [5]:
# Competition metric
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

# Prediction function (chose here which prediction strategy to use)
def prediction_function(pred,files,column_name,targets,tests_files):
    
    if pred == 'naive':
        # Naive prediction (persistence model)
        prediction = past_realized_volatility_per_stock(list_file=files,prediction_column_name=column_name)
        # Merge and evaluate results
        df_joined = train.merge(prediction[['row_id','pred']], on = ['row_id'], how = 'left')
        print(df_joined.head(5))

        # Estimate performances
        R2 = round(r2_score(y_true = df_joined['target'], y_pred = df_joined['pred']),3)
        RMSPE = round(rmspe(y_true = df_joined['target'], y_pred = df_joined['pred']),3)

        print('--')
        print(f'Performance of prediction: R2 score: {R2}, RMSPE: {RMSPE}')

    if pred == 'stupid_RF':
        # Stupid nonlinear regression between persistence and next volatility (random forest)
        prediction = stupidForestPrediction(list_file=files,prediction_column_name=column_name,targets=targets,tests_files=tests_files)
    
    return prediction

**Main evaluation code**

In [6]:
# Models
from persistence import *
#from custom1 import *

In [ ]:
# Glob book file train (contains all paths for each file in this folder)
list_order_book_file_train = glob.glob(r'C:\Users\laurent\ownCloud\Data\Kaggle\optiver-realized-volatility-prediction\book_train.parquet\*') # Comment the local path for online submission
list_order_book_file_test = glob.glob(r'C:\Users\laurent\ownCloud\Data\Kaggle\optiver-realized-volatility-prediction\book_test.parquet\*')

# Compute predictions
prediction = prediction_function(pred='naive',files=list_order_book_file_train,column_name='pred',targets=train,tests_files=list_order_book_file_test) 

In [ ]:
prediction

**Notes**

In [ ]:
from sklearn.ensemble import RandomForestRegressor

X = prediction['pred'].reshape(-1,1)
y = train['target'].reshape(-1,1)

regr = RandomForestRegressor(random_state=0)
regr.fit(X, y)
    